<a href="https://colab.research.google.com/github/bhartikaushal/home_sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,342 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,950 kB in 2s (1,318 kB/s)
Reading package li

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
sales_df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 2. Create a temporary view of the DataFrame.
sales_df.createOrReplaceTempView("home_sales")


In [22]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
start_time = time.time()
spark.sql("""SELECT YEAR(date), ROUND(AVG(price),2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY YEAR(date)
  ORDER BY YEAR(date) DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2022|296363.88|
|      2021|301819.44|
|      2020|298353.78|
|      2019| 300263.7|
+----------+---------+

--- 1.0103788375854492 seconds ---


In [23]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
start_time = time.time()
spark.sql("""SELECT YEAR(date), ROUND(AVG(price),2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY YEAR(date)
  ORDER BY YEAR(date) DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2022|292725.69|
|      2021|294211.46|
|      2020|294204.16|
|      2019|287287.82|
+----------+---------+

--- 1.0267987251281738 seconds ---


In [24]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
start_time = time.time()
spark.sql("""SELECT YEAR(date), ROUND(AVG(price),2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
  GROUP BY YEAR(date)
  ORDER BY YEAR(date) DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))




+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2022|290242.99|
|      2021|296330.96|
|      2020|292289.09|
|      2019|289859.14|
+----------+---------+

--- 1.6397595405578613 seconds ---


In [49]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""SELECT view, ROUND(AVG(price)) AS avg_price
  FROM home_sales
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
| 100|1026670.0|
|  51| 788128.0|
|  52| 733780.0|
|  53| 755215.0|
|  54| 798685.0|
|  55| 771153.0|
|  56| 718176.0|
|  57| 734341.0|
|  58| 759765.0|
|  59| 791453.0|
|  60| 754940.0|
|  61| 746878.0|
|  62| 759150.0|
|  63| 711615.0|
|  64| 767037.0|
|  65| 736680.0|
|  66| 712475.0|
|  67| 737971.0|
|  68| 716785.0|
|  69| 750538.0|
+----+---------+
only showing top 20 rows

--- 0.4164578914642334 seconds ---


In [37]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [38]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [52]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""SELECT view, ROUND(AVG(price)) AS avg_price
  FROM home_sales
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view
  """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
| 100|1026670.0|
|  51| 788128.0|
|  52| 733780.0|
|  53| 755215.0|
|  54| 798685.0|
|  55| 771153.0|
|  56| 718176.0|
|  57| 734341.0|
|  58| 759765.0|
|  59| 791453.0|
|  60| 754940.0|
|  61| 746878.0|
|  62| 759150.0|
|  63| 711615.0|
|  64| 767037.0|
|  65| 736680.0|
|  66| 712475.0|
|  67| 737971.0|
|  68| 716785.0|
|  69| 750538.0|
+----+---------+
only showing top 20 rows

--- 0.42812037467956543 seconds ---


In [53]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_df.write.partitionBy('date_built').parquet('p_home_sales',mode='overwrite')

In [54]:
# 11. Read the parquet formatted data.
p_sales_df = spark.read.parquet('p_home_sales')

In [55]:
# 12. Create a temporary table for the parquet data.
p_sales_df.createOrReplaceTempView('parquet_home_sales')

In [67]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""SELECT view, ROUND(AVG(price)) AS avg_price
  FROM home_sales
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view
  """).show(5)



print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
| 100|1026670.0|
|  51| 788128.0|
|  52| 733780.0|
|  53| 755215.0|
|  54| 798685.0|
+----+---------+
only showing top 5 rows

--- 0.39858174324035645 seconds ---


In [71]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [72]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('cached')
else:
  print('Uncached')


Uncached
